## Latent Personal Analysis in Python
by Uri Alon and Alex Abbey

This short tutorial will demonstrate how to use the Python implementation of LPA, as described in the [article](https://link.springer.com/article/10.1007/s11257-021-09295-7) in User Modeling and User-Adapted Interaction.

LPA helps analyze a corpus of text, or any set of data, by taking into account the missing popular elements and frequently used yet generally infrequent elements.

The first implementation for LPA was written by Hagit Ben-Shoshan in SQL and can be found [here](https://github.com/hagitbenshoshan/text_distance/) - 
For very large datasets, users of this package are encouraged to switch to the SQL implementation using cloud infrustructure, as the calculation of the results will be much faster.

Dependencies can be found in `requirements.txt`.

### Terminology

It can be useful to think of the basic unit of analysis as a word, which is part of a chapter, which is part of a book. To keep things general we'll use these terms:
* **Element** is the basic unit of analysis, i.e. a word
* **Documents** (or entities or categories) are compared to each other and in the context of their domain. **Local weight** is the normalized frequency of elements whithin a document. Each weighted entity's vector is termed Personal Vector Rates (**PVR**). 
* **Domain** (or world or corpus) consist of all of the categories, and is what the documents are compared to. The weighted domain vector is termed Domain Vector Rates (**DVR**).


In [1]:
import pandas as pd
import numpy as np
from LPA import Corpus

In this tutorial we demonstrate LPA using a small portion of the LOCO dataset (Miani, 2021), the unprocessed data available here: https://osf.io/snpcg/. The data you are using should consist of a count of elements per document (bag of words). For instance, with this dataset, each document is an article, each element is a (tokenized) word, and the frequency is a count of the amount of times the word appeared in an article. The input should be a csv file with the columns: `element`, `document`, `frequency_in_document`. 

In [2]:
freq = pd.read_csv('./frequency.csv')
display(freq.head())
display(freq.describe(include="all"))

,document,element,frequency_in_document
0,C00001,know,4.0
1,C00001,associ,5.0
2,C00001,said,1.0
3,C00001,known,3.0
4,C00001,colonel,1.0


,document,element,frequency_in_document
count,350874,350869,350874.000000
unique,1000,24709,NaN
top,C0029e,one,NaN
freq,2047,727,NaN
mean,NaN,NaN,1.829950
std,NaN,NaN,2.434249
min,NaN,NaN,1.000000
25%,NaN,NaN,1.000000
50%,NaN,NaN,1.000000
75%,NaN,NaN,2.000000


#### Creating the domain
Start LPA by instantiating a `Corpus` object. `Corpus` can be instantiated either with a frequency DataFrame, as in the following code cell, or by sending it two Series, one of all documents and one of all elements.

The domain (DVR) consists of the frequency of all elements from all categories together. If the `Corpus` wasn't instantiated with a frequency dataframe, you must use it as an argument for `create_dvr`. Use the `equally_weighted=True` to create a DVR where each document is equally weighted (fdefults to `False`). Use the head and tail of the `dvr` to calibrate your epsilon.

In [3]:
corpus = Corpus(freq=freq)
dvr = corpus.create_dvr()
dvr

,element,global_weight
0,one,0.004708
1,peopl,0.004643
2,us,0.004303
3,world,0.004249
4,use,0.003856
...,...,...
24704,nonstat,0.000002
24705,collegian,0.000002
24706,nonwhit,0.000002
24707,noonan,0.000002


#### Setting the value of $\varepsilon$
No $\varepsilon$ (`epsilon`) is set by default. $\varepsilon$ should be proportionate to the size of the domain, so a recommended value is 1 / (domain size * `epsilon_frac`). In this case if `epsilon_frac` is a number greater than 1 will decrease the weight of $\varepsilon$ (the default weight given to missing terms) while a number between 0 and 1 will increase it.

In [4]:
# pick epsilon
epsilon_frac=2
epsilon = 1/(len(dvr)*epsilon_frac)
epsilon

2.0235541705451456e-05

#### Creating Signatures
Another prominent use of LPA is creating a signature for every category, which is made up of the most meaningful terms for every category, whether in their prominence or absence. Usually, one won't need the full signature, but rather signatures of a certain length, as shown in the paper. To create signatures you can use the explicit process:
Create signatures with `create_signatures()`. This will give you a list of signatures (dataframes) with $KLD_\varepsilon$ distances of all of the words in the corpus for each category, and a list of dataframes of the n most significant elements in the whole corpus. 

Parameters:

* `overused` should stay `True` if you want to get positive and negative values, that signify the element is used more within the category (locally) than globally in the DVR,and the opposite, repectively.
* `most_significant` is the amount of most significant elements over the whole corpus you want to output (defaults to 30). Use `None` if most significant elements aren't needed.
* Use `sig_length` to cut the signatures to a certain length (defaults to 500), or use `None` if you don't want to cut the signatures at all.

In [2]:
signatures = corpus.create_signatures(epsilon=epsilon, most_significant=None)
signatures[0]

NameError: name 'lpa' is not defined

To get a feel for the dataset as a whole, check the the distance of each category from the domain by computing the summed distance of the signature from the `dvr`. 

In [2]:
distances = lpa.create_distances(freq)
distances.groupby("category")["KL"].sum().plot.hist()
# lpa.distance_summary(df).head()

NameError: name 'lpa' is not defined

#### Distances between pairs of categories (Sockpuppet Distance)
Finally, one can use the signatures created to calculate the L1 distance between every pair of categories. 
`sockpuppet_distance` accepts two DataFrames of $n$ and $m$ signatures respectively and compares them, returning a matrix sized $n \times m$. Different signature lengths and epsilons can have dramatic effects on the results.

In [27]:
spd = lpa.sockpuppet_distance(signatures1=cut.iloc[:5000], signatures2=cut.iloc[:5000])
spd

func: sockpuppet_distance took: 0:00:00.196844


category,C00001,C00003,C00004,C00005,C00007,C00008,C00009,C0000a,C0000b,C0000c
category,,,,,,,,,,
C00001,0.000000,2.757313,2.242467,1.517752,2.655146,2.481741,1.875268,2.315507,3.112205,2.193046
C00003,2.757313,0.000000,2.699781,2.860203,2.875945,2.760948,2.618410,2.701495,3.132657,2.691435
C00004,2.242467,2.699781,0.000000,2.287605,2.410529,2.430894,2.037306,2.364594,2.784505,2.134203
C00005,1.517752,2.860203,2.287605,0.000000,2.765364,2.519797,2.020111,2.381801,3.218368,2.241575
C00007,2.655146,2.875945,2.410529,2.765364,0.000000,2.545261,2.363791,2.668364,2.816953,2.258325
C00008,2.481741,2.760948,2.430894,2.519797,2.545261,0.000000,2.223212,2.463562,2.954522,2.373065
C00009,1.875268,2.618410,2.037306,2.020111,2.363791,2.223212,0.000000,2.243098,2.837591,1.972219
C0000a,2.315507,2.701495,2.364594,2.381801,2.668364,2.463562,2.243098,0.000000,3.025728,2.377948
C0000b,3.112205,3.132657,2.784505,3.218368,2.816953,2.954522,2.837591,3.025728,0.000000,2.736792


In [28]:
import altair as alt
df = spd.rename_axis("category_1").melt(ignore_index=False).reset_index()
alt.Chart(df).mark_rect().encode(
    x="category_1", y="category", color="value"
)

alt.Chart(...)

### Further analysis
Once we have calculated the distances between every category, we can perform further analysis on the results, for instance by clustering the sockpuppet distances and finding similar categories. 

## Good Luck!

### References
Miani, A., Hills, T. & Bangerter, A. LOCO: The 88-million-word language of conspiracy corpus. Behav Res (2021). https://doi.org/10.3758/s13428-021-01698-z

Mokryn, O., Ben-Shoshan, H. Domain-based Latent Personal Analysis and its use for impersonation detection in social media. User Model User-Adap Inter 31, 785–828 (2021). https://doi.org/10.1007/s11257-021-09295-7